# Conv_2D

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mlearn_cpu.functional as F
import mlearn_cpu as mlearn
import time
import torch
from time import time
import sys

In [2]:
_inputs = np.random.randn(32,3,128,128)
_w = np.random.randn(2,3,5,5)
_b = np.ones((2))
EPOCHS = 10

In [3]:
start = time()
for e in range(EPOCHS):
    out = F.conv_2d(_inputs,_w,_b,stride=2)
    print(f"\r{e+1}/{EPOCHS}",end="")
    sys.stdout.flush()
mlearn_conv = (time() - start)

10/10

In [4]:
inputs = torch.tensor(_inputs)
w = torch.tensor(_w)
b = torch.ones(2)
start = time()
for e in range(EPOCHS):
    torch.nn.functional.conv2d(inputs,w,b.double(),2)
    print(f"\r{e+1}/{EPOCHS}",end="")
    sys.stdout.flush()
torch_conv = (time() - start)

10/10

In [5]:
print("mlearn -> %.5f"%mlearn_conv)
print("torch -> %.5f"%torch_conv)
print("比Pytorch慢 %.5f倍"%(mlearn_conv/torch_conv))

mlearn -> 7.33051
torch -> 0.12432
比Pytorch慢 58.96586倍


In [9]:
nb.jit(nopython=True)
def conv_2d(inputs, weights,
              bias = None, stride: int = 1, padding: int = 0):

    out_x = (inputs.shape[2] - weights.shape[2] + 2 * padding) // stride + 1
    out_y = (inputs.shape[3] - weights.shape[3] + 2 * padding) // stride + 1

    if bias is None:
        bias = ones(weights.shape[0], 1, 1)
    else:
        if bias.shape[0] == weights.shape[0] and len(bias.shape) == 1:
            bias = bias.reshape(2, 1, 1)
        elif bias.shape == (weights.shape[0],1,1):
            pass

    view_shape = (1, out_x, out_y,
                  weights.shape[1], weights.shape[2], weights.shape[3])
    batch_result = []
    for sample in inputs:
        arr = np.lib.stride_tricks.as_strided(
            sample, view_shape, sample.strides*2).reshape(view_shape[1:])
        sample_result = []
        for out_dim in weights:
            single_channel_conv = []
            for row in arr:
                for col in row:
                    single_channel_conv.append((col * out_dim).sum())
            sample_result.append(
                np.array(single_channel_conv).reshape(out_x, out_y))
        sample_result = np.array(sample_result) + bias
        batch_result.append(sample_result)
    data = np.array(batch_result)